In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Acquisition


In [2]:
df = pd.read_csv('telco_churn_customers.csv')

In [3]:
def peekatdata(dataframe):
    head_df = df.head()
    tail_df = df.tail()
    head_tail = head_df.append(tail_df)
    print('Head and Tail\n\n', head_tail)
    print('--'*55)
    shape_tuple = df.shape
    print('Shape\n\n', shape_tuple)
    print('--'*20)
    describe_df = df.describe()
    print('Describe\n\n', describe_df)
    print('--'*20)
    print('Index\n\n', df.index)
    print('--'*20)
    print('Data Types\n\n', df.dtypes)
    print('--'*20)
    print('Null Value Count\n\n', df.isnull().sum())
    print('--'*55)
    print('Memory\n\n', df.memory_usage)

# Data Prep


In [4]:
def df_value_counts(df):
    for col in df.columns: 
        n = df[col].unique().shape[0] 
        col_bins = min(n,10) 
        if df[col].dtype in ['int64','float64'] and n > 10:
            print('%s:' % col)
            print(df[col].value_counts(bins=col_bins, sort=False)) 
        else: 
            print(df[col].value_counts()) 
        print('\n')


In [5]:
# Some of the total charges have empty space string ' ', replace with NaN 
df.replace(' ', np.nan, inplace=True)

# Replace NaN with 0
df.fillna(0, inplace=True)

# Turn the column from type object to float
df["total_charges"] = df.total_charges.astype(float)

In [6]:
# Transform churn such that "yes" = 1 and "no" = 0

def transform_churn(df):
    df['churn'] = df['churn'].replace({'Yes': 1, 'No': 0})
    return df

In [7]:
# Compute a new feature, tenure_year, that is a result of translating tenure from months to years.

def tenure_year(df):
    tenure_year = (df.tenure / 12)
    df['tenure_year']= tenure_year.round(2) # creates new column for above
    return df

In [8]:
def phone_info(df):
    df["phone_id"] = df["phone_service"].map(str) + df["multiple_lines"]
    df['phone_id'] = df['phone_id'].replace({'YesYes': 2, 'NoNo phone service': 0, 'YesNo': 1})
    return df


In [9]:
# Figure out a way to capture the information contained in dependents and partner into a single variable of dtype int.
# Transform the data and place in a new column household_type_id.

def household_type_id(df):
    df["household_type_id"] = df["partner"].map(str) + df["dependents"]
    df['household_type_id'] = df['household_type_id'].replace({'YesYes': 3, 'NoNo': 0, 'YesNo': 2, 'NoYes': 1})
    return df


In [10]:
# Figure out a way to capture the information contained in streaming_tv and streaming_movies into a single
# variable of dtype int. Transform the data and place in a new column streaming_services.

def streaming_services(df):
    df["streaming_services"] = df["partner"].map(str) + df["dependents"]
    df['streaming_services'] = df['streaming_services'].replace({'YesYes': 3, 'NoNo': 0, 'YesNo': 2, 'NoYes': 1})
    return df



In [11]:
def online_security_info(df):
    df["online_security"].map(str) + df["online_backup"]
    df['online_security'] = df['online_security'].replace({'No internet serviceNo internet service': 0,
                                                           'NoNo': 1, 
                                                           'NoYes': 2,
                                                           'YesNo': 3,
                                                           'YesYes': 4})
    return df


In [12]:
X = df.drop('churn', axis=1)
y = df[['churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)


In [13]:
encoded = df[['gender', 'device_protection', 'tech_support', 'paperless_billing']].apply(LabelEncoder().fit_transform)
df.drop(columns= ['gender', 'device_protection', 'tech_support', 'paperless_billing'], axis=1, inplace=True)
df = df.join(encoded)




In [14]:
df.head()

,customer_id,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,...,streaming_movies,contract_type_id,payment_type_id,monthly_charges,total_charges,churn,gender,device_protection,tech_support,paperless_billing
0,0002-ORFBO,0,Yes,Yes,9,Yes,No,1,No,Yes,...,No,2,2,65.6,593.30,No,0,0,2,1
1,0003-MKNFE,0,No,No,9,Yes,Yes,1,No,No,...,Yes,1,2,59.9,542.40,No,1,0,0,0
2,0004-TLHLJ,0,No,No,4,Yes,No,2,No,No,...,No,1,1,73.9,280.85,Yes,1,2,0,1
3,0011-IGKFF,1,Yes,No,13,Yes,No,2,No,Yes,...,Yes,1,1,98.0,1237.85,Yes,1,2,0,1
4,0013-EXCHZ,1,Yes,No,3,Yes,No,2,No,No,...,No,1,2,83.9,267.40,Yes,0,0,2,1


In [15]:
df.columns

Index(['customer_id', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'multiple_lines', 'internet_service_type_id',
       'online_security', 'online_backup', 'streaming_tv', 'streaming_movies',
       'contract_type_id', 'payment_type_id', 'monthly_charges',
       'total_charges', 'churn', 'gender', 'device_protection', 'tech_support',
       'paperless_billing'],
      dtype='object')